In [ ]:
!git clone https://github.com/dsa-playground/tvt2023.git
%cd /content/tvt2023/
!git pull
!pip install -r requirements.txt -t "tvt2023"

# Inleiding

### Use case
Titanic algemeen, kans op overleven, maar zou jij het overleefd hebben?!
Met Machine Learning kun je middels algoritmes bepalen of iemand wel of niet overleefd heeft gezien de beschikbare variabelen.  

### Instructies omgeving
Stukje over Google Colab met de belangrijkste toets-combinaties / ... 

### Imports & settings
Importeer de functies benodigd voor de training (of... dit gebeurd in cellen vooraf door cursusleider)

In [ ]:
# Imports
import pandas as pd
from scripts.preprocess.preprocess_frontend import zie_settings, laden_data, opschonen_data, voeg_passagiers_toe
# from scripts.preprocess.preprocess_clean import rename_cols, change_order_cols, drop_cols, replace_values_cols, \
#     floor_values_cols, fillna_cols, dropna_rows, create_index

# from scripts.preprocess.preprocess_collect import add_record, add_multiple_records

# Settings
# settings for pandas
pd.set_option("display.max.columns",None) # alle kolommen tonen
pd.set_option("display.max.rows",500)    # eerste 500 rijen tonen
pd.set_option("display.precision", 2)     # precisie van de kolommen aanpassen
pd.set_option('display.float_format', lambda x: '{:.3f}'.format(x)) # floats output tot 3 decimalen
pd.set_option('display.max_colwidth', None)

In [ ]:
_config = zie_settings()

Toelichting Data Science, train/test principe. 

Laten we eens gaan kijken naar de dataset voor het trainen en testen van een model...


In [ ]:
df_train, df_test = laden_data()
display(df_train.head(), df_test.head())

In [ ]:
df_train_clean, df_test_clean = opschonen_data(df_train, df_test)
display(df_train_clean.head(), df_test_clean.head())

Toelichting dataset (naslagwerk):
...

Laten we onszelf toevoegen aan de dataset.

Mogelijke vraag (om te kijken of ze opletten/begrepen hebben): Welke dataset zou dat zijn?!


In [ ]:
# df_train, df_test = voeg_passagiers_toe()
df_train_extended, df_test_extended = voeg_passagiers_toe(df_train_clean, df_test_clean)

In [ ]:
df_train_extended.tail()

In [ ]:
df_test_extended.tail()

In [ ]:
_config['preprocess']['data']['collect']['transform_multi']

toevoegen_passagiers() # pop-up venster?

EDA

Toelichting trainen / algoritme?
Iets over classificatie algoritme (x aantal opties). Nu zijn immers 2 opties: Wel / niet overleven. 

Willen we nog iets doen met train_test_split? Zo ja, dan test set hernoemen in tekst?

Trainen van model / evaluatie?

Voorspellen testset met deelnemers training + visualisaties